In [1]:
import pandas as pd

In [31]:
df = pd.read_csv("../../DATASETS_TRACTATS/TOT.csv")

In [32]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})
df["P/S"] = df["P/S"].str.replace(",",".").astype(float)

In [33]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "ALTITUD", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [34]:
#df = df[df["CAMPAÑA"] != "22"].drop(columns =["SUPERFICIE"])

In [35]:
df = df[df["CAMPAÑA"].isin(["22", "14", "15"]) == False].drop(columns =["SUPERFICIE"])

In [36]:
X = df.drop(columns = ["PRODUCCION", "CAMPAÑA"])
y = df["PRODUCCION"]

In [37]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,P/S
2264,16,913,105,19,76,2,1,1,1,22413.779,600.000000,0.00000,6136.817117
2265,16,914,105,19,76,2,1,1,1,44076.221,600.000000,0.00000,6136.817117
2266,16,7,4,2,3,3,2,1,1,16900.000,654.501718,9.19244,6136.817117
2267,16,8,5,2,3,4,2,1,1,7320.000,654.501718,9.19244,6136.817117
2268,16,12,7,2,3,5,2,1,1,11294.400,650.000000,10.00000,6136.817117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,21,909,104,18,6,8,1,1,1,28160.100,700.000000,0.00000,5199.427417
8522,21,911,104,18,6,2,1,1,1,41310.000,700.000000,0.00000,5199.427417
8523,21,911,104,18,6,5,1,1,1,45420.000,700.000000,0.00000,5199.427417
8524,21,912,104,18,6,6,1,1,1,56140.000,700.000000,0.00000,5199.427417


#### Random

In [49]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid_random(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [50,60,80],
                            "learning_rate": [0.15],
                            "max_depth": [40, 30],
                            "min_child_weight": [1, 2, 4],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart", "gbtree"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

In [50]:
model_random = grid_random(X,y)

In [40]:
pdd = pd.DataFrame(model_random.cv_results_)

In [51]:
pdd

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,...,std_test_r2,rank_test_r2,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error
0,3.595193,0.015854,0.075801,0.002713,dart,0.4,0.1,0.15,40,1,...,0.072438,2,-8789.294074,-5606.297753,-6766.711749,-6517.083619,-7058.232222,-6947.523883,1041.271641,2
1,5.833537,0.365066,0.118200,0.030967,dart,0.4,0.1,0.15,40,1,...,0.081960,11,-9077.869118,-5652.910765,-6723.433629,-6741.834103,-7332.056153,-7105.620754,1125.149825,11
2,17.352502,0.653792,0.245999,0.010602,dart,0.4,0.1,0.15,40,1,...,0.090259,19,-9362.494169,-5787.299909,-6744.894447,-7044.031598,-7614.378147,-7310.619654,1184.404956,19
3,33.458318,0.699297,0.308386,0.006814,dart,0.4,0.1,0.15,40,1,...,0.091788,27,-9416.737023,-5819.563153,-6755.418429,-7133.597829,-7715.890086,-7368.241304,1195.508916,27
4,5.882605,0.036736,0.132016,0.017144,dart,0.4,0.1,0.15,60,1,...,0.072372,1,-8786.764352,-5600.410706,-6773.426492,-6510.358015,-7053.535307,-6944.898974,1042.127371,1
5,9.048246,0.091560,0.163210,0.015152,dart,0.4,0.1,0.15,60,1,...,0.082032,12,-9078.580164,-5639.943517,-6736.373982,-6758.800820,-7332.306658,-7109.201028,1126.829912,12
6,19.696066,0.101498,0.220630,0.012993,dart,0.4,0.1,0.15,60,1,...,0.090087,20,-9360.501020,-5786.778041,-6753.358064,-7045.546137,-7614.895162,-7312.215685,1183.003142,20
7,34.015432,0.312778,0.319914,0.019321,dart,0.4,0.1,0.15,60,1,...,0.091637,28,-9415.232751,-5818.578414,-6763.877309,-7135.526919,-7718.170596,-7370.277198,1194.443022,28
8,4.255662,0.027616,0.118800,0.006912,dart,0.2,0.1,0.15,40,1,...,0.047446,47,-9273.927815,-6807.564761,-8551.103156,-7369.067260,-8422.409460,-8084.814490,881.935026,47
9,6.486265,0.097627,0.148000,0.011190,dart,0.2,0.1,0.15,40,1,...,0.051169,37,-9021.440536,-6418.864787,-8086.760271,-7073.850368,-8093.266865,-7738.836565,902.914927,41


In [124]:
pdd.iloc[:,15:20]

,split3_test_r2,split4_test_r2,mean_test_r2,std_test_r2,rank_test_r2
0,0.802515,0.717459,0.754676,0.051602,1


#### Testejant un any

In [125]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid_test_year(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

In [126]:
years_to_test = df["CAMPAÑA"].unique()
model_one_year_test = {}
for year in years_to_test:
    X_train = X[df["CAMPAÑA"] != year]
    y_train = y[df["CAMPAÑA"] != year]
    X_test = X[df["CAMPAÑA"] == year]
    y_test = y[df["CAMPAÑA"] == year]
    m_y = grid_test_year(X_train, y_train)
    score_y = m_y.score(X_test, y_test)
    model_one_year_test[year] = (m_y, score_y)

In [127]:
for year in years_to_test:
    print(year)
    print(pd.DataFrame(model_one_year_test[year][0].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])
    print(model_one_year_test[year][1])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.770077                           -6400.227166
-6970.362242533284
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.742365                           -6505.414175
-6363.449216466493
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.754423                           -6421.569045
-6511.040233589743
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.747729                           -6666.082963
-5547.49357935647
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.737162                           -6639.049979
-6019.240218921513
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.734678                           -6866.481778
-4667.7052486376015
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.754803                           -6486.482451
-6928.485543134875
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      0.752088   

In [90]:
pdd2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,...,std_test_r2,rank_test_r2,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error
0,7.909569,0.08696,0.124994,0.007575,dart,0.4,0.1,0.15,40,1,...,0.051466,1,-7120.566775,-6517.305645,-5851.314963,-6024.383185,-6978.813926,-6498.476899,502.326073,1


#### Per anys

In [98]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid_one_year(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBRegressor(),
                param_grid={"n_estimators": [100],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.1],
                            "colsample_bytree":[0.4],
                            "booster": ["dart"]},
                cv=5, scoring=["r2", "neg_root_mean_squared_error"], refit = "neg_root_mean_squared_error", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    
    return gsc

In [99]:
years_to_test2 = df["CAMPAÑA"].unique()
model_one_year = {}
for year in years_to_test2:
    X2 = X[df["CAMPAÑA"] == year]
    y2 = y[df["CAMPAÑA"] == year]
    m_y = grid_one_year(X2, y2)
    model_one_year[year] = m_y

In [101]:
for year in years_to_test:
    print(year)
    print(pd.DataFrame(model_one_year[year].cv_results_).loc[:,["mean_test_r2", "mean_test_neg_root_mean_squared_error"]])

14
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.580223                          -13356.864576
15
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.121736                          -14498.252638
16
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      -0.17357                          -13862.194447
17
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.084617                          -11088.234603
18
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.200754                          -14500.058328
19
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.131728                          -11798.222292
20
   mean_test_r2  mean_test_neg_root_mean_squared_error
0     -0.423209                           -13884.71261
21
   mean_test_r2  mean_test_neg_root_mean_squared_error
0      -0.49595                          -14530.304329
